Simply run the code on google colab or on juypter notebook to see the results.

In [100]:
import requests
import pandas as pd
import time
import numpy as np

The reason why I have used ccxt instead of binance is beacuse the binance has blocked access from our region.


---

I got this reply after running the code for getting the data

Error: 451
"Service unavailable from a restricted location...


In [101]:
def fetch_cryptocompare_ohlc(symbol='BTC', tsym='USDT', limit=2000, total=5000):
    base_url = 'https://min-api.cryptocompare.com/data/v2/histominute'
    all_data = []
    toTs = int(time.time())

    while len(all_data) < total:
        batch_limit = min(limit, total - len(all_data)) - 1
        params = {
            'fsym': symbol,
            'tsym': tsym,
            'limit': batch_limit,
            'toTs': toTs
        }
        response = requests.get(base_url, params=params)
        data = response.json()

        if data['Response'] != 'Success':
            raise Exception(f"API Error: {data}")

        batch = data['Data']['Data']
        if not batch:
            break

        all_data = batch + all_data
        toTs = batch[0]['time'] - 1
        time.sleep(0.25)

    df = pd.DataFrame(all_data)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)

    df['time'] = df.index

    df = df[['time', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']]

    return df

# Usage
df = fetch_cryptocompare_ohlc()
print(df.tail())

                                   time       open       high        low  \
time                                                                       
2025-06-03 18:13:00 2025-06-03 18:13:00  105649.38  105693.07  105649.38   
2025-06-03 18:14:00 2025-06-03 18:14:00  105686.84  105686.84  105662.39   
2025-06-03 18:15:00 2025-06-03 18:15:00  105664.15  105664.15  105613.79   
2025-06-03 18:16:00 2025-06-03 18:16:00  105625.56  105636.97  105599.87   
2025-06-03 18:17:00 2025-06-03 18:17:00  105605.57  105614.85  105605.57   

                         close  volumefrom    volumeto  
time                                                    
2025-06-03 18:13:00  105686.84       20.19  2133712.16  
2025-06-03 18:14:00  105664.15       15.44  1631230.60  
2025-06-03 18:15:00  105625.56       33.74  3563684.41  
2025-06-03 18:16:00  105605.57       25.33  2674840.66  
2025-06-03 18:17:00  105614.85        0.00        0.00  


In [102]:
df.columns

Index(['time', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto'], dtype='object')

In [103]:
def real_time_rsi_ema_strategy(df):
    N_EMA = 21
    N_RSI = 14
    alpha_ema = 2 / (N_EMA + 1)
    ema_list = [np.nan] * len(df)
    rsi_list = [np.nan] * len(df)
    gain_list = []
    loss_list = []
    in_position = False
    trades = []
    ema_prev = None
    avg_gain = None
    avg_loss = None

    for i in range(len(df)):
        close = df.iloc[i]['close']

        # === EMA Calculation ===
        if i >= N_EMA - 1:
            if i == N_EMA - 1:
                sma = df.iloc[i - N_EMA + 1:i + 1]['close'].mean()
                ema = sma
            else:
                ema = alpha_ema * close + (1 - alpha_ema) * ema_prev
            ema_list[i] = ema
            ema_prev = ema

        # === RSI Calculation ===
        if i == 0:
            change = np.nan
        else:
            change = close - df.iloc[i - 1]['close']

        gain = max(change, 0) if not np.isnan(change) else 0
        loss = -min(change, 0) if not np.isnan(change) else 0
        gain_list.append(gain)
        loss_list.append(loss)

        if i >= N_RSI:
            if i == N_RSI:
                avg_gain = np.mean(gain_list[i - N_RSI + 1:i + 1])
                avg_loss = np.mean(loss_list[i - N_RSI + 1:i + 1])
            else:
                avg_gain = (avg_gain * (N_RSI - 1) + gain) / N_RSI
                avg_loss = (avg_loss * (N_RSI - 1) + loss) / N_RSI

            rs = avg_gain / avg_loss if avg_loss != 0 else np.inf
            rsi = 100 - (100 / (1 + rs))
            rsi_list[i] = rsi

        # === Trading Logic ===
        if i < max(N_EMA, N_RSI):
            continue

        rsi_val = rsi_list[i]
        ema_val = ema_list[i]
        time_val = df.iloc[i]['time']

        if pd.isna(rsi_val) or pd.isna(ema_val):
            continue

        # Entry Condition
        if not in_position:
            if rsi_val >= 30 and close > ema_val:
                entry_price = close
                entry_time = time_val
                in_position = True

        # Exit Condition
        elif in_position:
            if rsi_val >= 70 or close < ema_val:
                exit_price = close
                exit_time = time_val
                pnl = exit_price - entry_price
                trades.append({
                    'Entry Time': entry_time,
                    'Entry Price': entry_price,
                    'Exit Time': exit_time,
                    'Exit Price': exit_price,
                    'Strategy': 'RSI_EMA',
                    'PnL': pnl,
                    'Status': 'Closed'
                })
                in_position = False

    df['EMA_21'] = ema_list
    df['RSI_14'] = rsi_list

    # Return updated DataFrame and trade log
    return df, pd.DataFrame(trades)

In [104]:
df, trade_summary = real_time_rsi_ema_strategy(df)
print(trade_summary)

             Entry Time  Entry Price           Exit Time  Exit Price Strategy  \
0   2025-05-31 07:31:00    103736.82 2025-05-31 07:37:00   103739.18  RSI_EMA   
1   2025-05-31 07:42:00    103742.40 2025-05-31 07:43:00   103703.24  RSI_EMA   
2   2025-05-31 07:52:00    103723.92 2025-05-31 08:07:00   103720.16  RSI_EMA   
3   2025-05-31 08:23:00    103643.09 2025-05-31 08:34:00   103665.97  RSI_EMA   
4   2025-05-31 08:46:00    103622.20 2025-05-31 08:59:00   103652.30  RSI_EMA   
..                  ...          ...                 ...         ...      ...   
467 2025-06-03 16:31:00    106397.07 2025-06-03 16:36:00   106375.89  RSI_EMA   
468 2025-06-03 17:10:00    105998.09 2025-06-03 17:11:00   105961.00  RSI_EMA   
469 2025-06-03 17:18:00    105972.59 2025-06-03 17:30:00   105926.23  RSI_EMA   
470 2025-06-03 17:51:00    105915.36 2025-06-03 17:52:00   105862.51  RSI_EMA   
471 2025-06-03 17:55:00    105941.88 2025-06-03 18:01:00   105841.94  RSI_EMA   

       PnL  Status  
0     

In [105]:
def real_time_ema_crossover(df):
    N1 = 5
    N2 = 20
    alpha_5 = 2 / (N1 + 1)
    alpha_20 = 2 / (N2 + 1)

    ema5_list = [np.nan] * len(df)
    ema20_list = [np.nan] * len(df)

    trades = []
    in_position = False
    entry_time = entry_price = None

    ema5_prev = None
    ema20_prev = None

    for i in range(len(df)):
        close = df.iloc[i]['close']

        # === EMA-5 Calculation ===
        if i >= N1 - 1:
            if i == N1 - 1:
                ema5 = df.iloc[i - N1 + 1:i + 1]['close'].mean()
            else:
                ema5 = alpha_5 * close + (1 - alpha_5) * ema5_prev
            ema5_list[i] = ema5
            ema5_prev = ema5

        # === EMA-20 Calculation ===
        if i >= N2 - 1:
            if i == N2 - 1:
                ema20 = df.iloc[i - N2 + 1:i + 1]['close'].mean()
            else:
                ema20 = alpha_20 * close + (1 - alpha_20) * ema20_prev
            ema20_list[i] = ema20
            ema20_prev = ema20

        # Skip till both EMAs are available
        if i < N2:
            continue

        prev_ema5 = ema5_list[i - 1]
        prev_ema20 = ema20_list[i - 1]
        curr_ema5 = ema5_list[i]
        curr_ema20 = ema20_list[i]

        if any(pd.isna([prev_ema5, prev_ema20, curr_ema5, curr_ema20])):
            continue

        time_val = df.iloc[i]['time']

        # === Entry Condition ===
        if not in_position and (prev_ema5 < prev_ema20) and (curr_ema5 > curr_ema20):
            in_position = True
            entry_time = time_val
            entry_price = close

        # === Exit Condition ===
        elif in_position and (prev_ema5 > prev_ema20) and (curr_ema5 < curr_ema20):
            exit_time = time_val
            exit_price = close
            pnl = exit_price - entry_price
            trades.append({
                'Entry Time': entry_time,
                'Entry Price': entry_price,
                'Exit Time': exit_time,
                'Exit Price': exit_price,
                'Strategy': 'MACD_EMA_Crossover',
                'PnL': pnl,
                'Status': 'Closed'
            })
            in_position = False
            entry_time = entry_price = None

    # If still in position at the end
    if in_position:
        trades.append({
            'Entry Time': entry_time,
            'Entry Price': entry_price,
            'Exit Time': None,
            'Exit Price': None,
            'Strategy': 'MACD_EMA_Crossover',
            'PnL': None,
            'Status': 'Open'
        })

    df['EMA_5'] = ema5_list
    df['EMA_20'] = ema20_list

    return df, pd.DataFrame(trades)

In [106]:
df, trade_summary = real_time_ema_crossover(df)
print(trade_summary)

             Entry Time  Entry Price           Exit Time  Exit Price  \
0   2025-05-31 07:32:00    103755.47 2025-05-31 07:41:00   103730.21   
1   2025-05-31 07:42:00    103742.40 2025-05-31 07:43:00   103703.24   
2   2025-05-31 07:54:00    103821.30 2025-05-31 08:08:00   103738.21   
3   2025-05-31 08:25:00    103698.47 2025-05-31 08:36:00   103604.40   
4   2025-05-31 08:47:00    103684.93 2025-05-31 09:01:00   103646.44   
..                  ...          ...                 ...         ...   
144 2025-06-03 15:35:00    106665.08 2025-06-03 15:46:00   106588.84   
145 2025-06-03 15:57:00    106641.11 2025-06-03 16:01:00   106512.23   
146 2025-06-03 16:16:00    106511.57 2025-06-03 16:18:00   106427.97   
147 2025-06-03 17:20:00    106050.85 2025-06-03 17:30:00   105926.23   
148 2025-06-03 17:55:00    105941.88 2025-06-03 18:01:00   105841.94   

               Strategy     PnL  Status  
0    MACD_EMA_Crossover  -25.26  Closed  
1    MACD_EMA_Crossover  -39.16  Closed  
2    MACD